In [2]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Conv2D
from tensorflow.keras.models import load_model, Model

In [7]:
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

import yolo_utils

In [11]:
a = np.random.randn(19*19,5,1) #p_c
b = np.random.randn(19*19,5,80) #c_1 ~ c_80
c = a * b #计算后的维度将会是(19x19,5,80)
index = c.argmax(axis=-1)
print(index)
print(index.shape)

[[67 73 29 75 14]
 [31 11  3 19 64]
 [73 11 17  9 72]
 ...
 [70 36 74 15 52]
 [14 24 53 37  5]
 [66 77 46 16 58]]
(361, 5)


In [ ]:
# 分类阈值过滤
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=0.6):
    """
    通过阈值来过滤对象和分类的置信度。
    Args:
        box_confidence: tensor类型，维度为（19,19,5,1）,包含19x19单元格中每个单元格预测的5个锚框中的所有的锚框的pc （一些对象的置信概率）。
        boxes: tensor类型，维度为(19,19,5,4)，包含了所有的锚框的（px,py,ph,pw ）。
        box_class_probs: tensor类型，维度为(19,19,5,80)，包含了所有单元格中所有锚框的所有对象( c1,c2,c3，···，c80 )检测的概率。
        threshold: 实数，阈值，如果分类预测的概率高于它，那么这个分类预测的概率就会被保留。

    Returns:
        scores - tensor 类型，维度为(None,)，包含了保留了的锚框的分类概率。
        boxes - tensor 类型，维度为(None,4)，包含了保留了的锚框的(b_x, b_y, b_h, b_w)
        classes - tensor 类型，维度为(None,)，包含了保留了的锚框的索引
    Notes："None"是因为你不知道所选框的确切数量，因为它取决于阈值。
          比如：如果有10个锚框，scores的实际输出大小将是（10,）
    """
    # step1: 计算锚框的得分
    box_scores = box_confidence * box_class_probs

    # step2: 找到最大值的锚框的索引以及对应的最大值的锚框的分数
    box_classes = K.argmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1)

    # step3: 根据阈值创建掩码
    filtering_mask = (box_class_scores >= threshold) # 如执行下列操作：[0.9, 0.3, 0.4, 0.5, 0.1] < 0.4，返回的是[False, True, False, False, True]

    # 对scores, boxes 以及 classes使用掩码
    scores = tf.boolean_mask()